In [1]:
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType,DecimalType,DoubleType,BooleanType,TimestampType
from pyspark.shell import spark

from pyspark.sql.functions import initcap, lower,col,year,month,dayofmonth,expr
from pyspark.sql import functions as sf
from pyspark.sql.functions import regexp_replace
import datetime
import pandas as pd
import numpy as np
import re
 

#import org.apache.spark.sql.functions.format_number




os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark_app = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()







Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.9.12 (main, Apr  4 2022 05:22:27)
Spark context Web UI available at http://LAPTOP-PSIMHJKQ:4043
Spark context available as 'sc' (master = local[*], app id = local-1661364554350).
SparkSession available as 'spark'.


# Functional Requirement 1.1
1. Functional Requirements - Load Credit Card Database - cdw_sapp_branch (SQL)


In [3]:
#LOAD TABLE CDW_SAPP_D_BRANCH
df_branch = spark.read.format('org.apache.spark.sql.json').load('C:/Users/asha_/dev/capstoneproj/Data/cdw_sapp_branch.json')
df_branch.show(5)

+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|    Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL| Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|      14th Street|     19406|2018-04-18T16:51:...|
+-----------------+-----------+-

In [4]:
#Change datatype of branch columns
df_branch= df_branch.withColumn("BRANCH_CODE",col("BRANCH_CODE").cast("int"))
df_branch = df_branch.withColumn("BRANCH_ZIP",col("BRANCH_ZIP").cast("int"))
df_branch= df_branch.withColumn("LAST_UPDATED",col("LAST_UPDATED").cast("timestamp"))


df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: integer (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: integer (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)



In [6]:
#Replace 999999 for null on branch_zip column 
df_branch = df_branch.fillna({"BRANCH_ZIP": 999999})
df_branch.show(5)


+-----------------+-----------+------------+------------+------------+-------------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+-------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18 15:51:47|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18 15:51:47|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18 15:51:47|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18 15:51:47|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18 15:51:47|
|         Paterson|     

In [7]:
#df_branch.filter(df_branch.BRANCH_ZIP == 999999).show(truncate=False)

+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
|BRANCH_CITY|BRANCH_CODE|BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|BRANCH_ZIP|LAST_UPDATED|
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+



In [5]:
df_branch = df_branch.withColumn('BRANCH_PHONE',regexp_replace('BRANCH_PHONE',r'(^\d{3})(\d{3})(\d{4})',r'($1)$2-$3'))
df_branch.show(3)

+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 15:51:47|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 15:51:47|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|    Warren Street|     11419|2018-04-18 15:51:47|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
only showing top 3 rows



# Function Requirement 1.2

Data loading into Database

In [7]:
df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone_") \
.option("dbtable", "creditcard_capstone_.CDW_SAPP_BRANCH") \
.option("user", "root") \
.option("password", "a") \
.save()

# 2. Functional Requirements - Application Front-End
   2.1 Transaction Details Module